In [3]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE

#load dataset
train_features_data=pd.read_csv('train_features.csv')
test_features_data=pd.read_csv('test_features.csv')
labels_data=pd.read_csv('train_labels.csv')
#extract features with pandas groupby method

train_data=pd.DataFrame(train_features_data.groupby(['pid']).mean())
test_data=pd.DataFrame(test_features_data.groupby(['pid']).mean())
train=train_data.reset_index().drop(columns=['pid']).fillna(train_data.mean()) #assumed that time plays no role on predictions
test=test_data.reset_index().drop(columns=['pid']).fillna(test_data.mean())
display(train)
display(test)
#labels=labels_data.drop(columns=['pid'])
labels1 = labels_data[['LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total', 'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2', 'LABEL_Bilirubin_direct', 'LABEL_EtCO2']]
for label in ['LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total', 'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2', 'LABEL_Bilirubin_direct', 'LABEL_EtCO2']:
    print(label, (labels1[label] == 1).sum())
labels2 = labels_data[['LABEL_Sepsis']]
print('LABEL_Sepsis', (labels2['LABEL_Sepsis'] == 1).sum())
labels3 = labels_data[['LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']]
display(labels1['LABEL_BaseExcess'][0:50])

,Time,Age,EtCO2,PTT,BUN,Lactate,Temp,Hgb,HCO3,BaseExcess,...,Alkalinephos,SpO2,Bilirubin_direct,Chloride,Hct,Heartrate,Bilirubin_total,TroponinI,ABPs,pH
0,8.5,34.0,32.829817,38.801443,12.000000,2.383041,36.750000,8.566667,25.333333,-0.666667,...,96.675671,100.000000,1.170195,112.000000,23.200000,77.083333,1.544023,6.414359,114.500000,7.370000
1,6.5,86.0,32.829817,31.800000,32.000000,2.383041,36.000000,13.100000,23.801594,-0.789992,...,96.675671,96.000000,1.170195,105.925903,40.200000,59.000000,1.544023,0.440000,131.181818,7.375355
2,6.5,66.0,32.829817,34.600000,8.000000,2.383041,36.666667,10.550000,23.801594,-0.789992,...,130.000000,99.272727,0.100000,105.925903,33.550000,72.545455,0.600000,0.020000,125.454545,7.375355
3,7.5,66.0,32.829817,53.800000,32.000000,1.800000,37.166667,10.316667,19.500000,-2.857143,...,96.675671,99.333333,1.170195,113.500000,30.390000,87.333333,1.544023,6.414359,100.666667,7.352857
4,6.5,42.0,32.829817,38.801443,18.000000,2.383041,36.000000,10.748832,23.801594,-0.789992,...,96.675671,97.800000,1.170195,105.925903,32.005177,81.181818,1.544023,0.080000,204.545455,7.375355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18990,6.5,52.0,32.829817,25.800000,11.000000,1.700000,36.000000,9.500000,22.666667,0.000000,...,96.675671,96.181818,1.170195,106.000000,31.560000,108.272727,1.544023,6.414359,87.333333,7.330000
18991,6.5,66.0,32.829817,38.801443,33.000000,2.383041,37.500000,11.200000,23.801594,-0.789992,...,96.675671,96.090909,1.170195,105.925903,34.600000,92.909091,1.544023,6.414359,165.000000,7.375355
18992,6.5,44.0,32.829817,38.801443,15.000000,2.383041,38.000000,12.400000,24.000000,-3.500000,...,96.675671,100.000000,1.170195,97.000000,36.700000,99.100000,1.544023,6.414359,100.300000,7.300000
18993,6.5,70.0,32.829817,38.801443,23.000573,2.383041,36.500000,10.748832,23.801594,-0.789992,...,96.675671,99.100000,1.170195,105.925903,32.005177,64.400000,1.544023,6.414359,110.500000,7.375355


,Time,Age,EtCO2,PTT,BUN,Lactate,Temp,Hgb,HCO3,BaseExcess,...,Alkalinephos,SpO2,Bilirubin_direct,Chloride,Hct,Heartrate,Bilirubin_total,TroponinI,ABPs,pH
0,6.5,39.0,32.509552,41.350000,18.500000,2.322879,36.000000,9.650000,14.500000,-6.500000,...,114.000000,100.000000,1.131093,100.000000,28.450000,84.500000,24.100000,6.412313,128.625000,7.370000
1,6.5,84.0,32.509552,38.415921,23.056558,2.322879,35.000000,10.780526,23.808477,-0.634898,...,98.404732,97.000000,1.131093,106.004317,32.107626,99.833333,1.521280,6.412313,113.000000,7.376448
2,6.5,62.0,32.509552,38.415921,23.056558,2.322879,37.000000,10.780526,23.808477,-0.634898,...,98.404732,95.909091,1.131093,106.004317,32.107626,65.272727,1.521280,6.412313,106.800000,7.376448
3,6.5,71.0,32.509552,75.766667,21.333333,2.322879,36.400000,8.200000,24.666667,0.500000,...,113.666667,99.636364,1.250000,100.666667,22.990000,98.272727,2.033333,6.412313,129.181818,7.380000
4,6.5,51.0,32.509552,35.400000,47.000000,2.322879,36.500000,10.300000,22.000000,-0.634898,...,98.404732,95.818182,1.131093,103.000000,33.700000,93.727273,1.521280,6.412313,127.500000,7.376448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12659,6.5,29.0,32.509552,38.415921,23.056558,2.322879,37.000000,10.780526,23.808477,-0.634898,...,98.404732,97.272727,1.131093,106.004317,32.107626,71.090909,1.521280,6.412313,106.818182,7.376448
12660,6.5,83.0,32.509552,28.800000,22.000000,2.166667,36.333333,12.433333,24.000000,-3.000000,...,64.000000,97.727273,1.131093,106.000000,36.600000,90.363636,0.400000,6.412313,118.545455,7.273333
12661,6.5,74.0,32.509552,32.400000,38.500000,2.322879,35.500000,10.450000,31.500000,4.666667,...,98.404732,100.000000,1.131093,98.500000,29.850000,85.727273,1.521280,6.412313,115.666667,7.373333
12662,6.5,40.0,32.509552,38.415921,11.000000,2.322879,37.000000,11.050000,23.808477,-0.634898,...,98.404732,98.363636,1.131093,106.004317,33.500000,131.545455,1.521280,0.010000,130.300000,7.376448


LABEL_BaseExcess 5096
LABEL_Fibrinogen 1400
LABEL_AST 4554
LABEL_Alkalinephos 4487
LABEL_Bilirubin_total 4570
LABEL_Lactate 3803
LABEL_TroponinI 1895
LABEL_SaO2 4439
LABEL_Bilirubin_direct 644
LABEL_EtCO2 1254
LABEL_Sepsis 1088


0     1.0
1     0.0
2     1.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     1.0
9     1.0
10    0.0
11    0.0
12    1.0
13    0.0
14    0.0
15    1.0
16    0.0
17    0.0
18    0.0
19    0.0
20    0.0
21    1.0
22    1.0
23    0.0
24    0.0
25    0.0
26    0.0
27    1.0
28    0.0
29    0.0
30    0.0
31    1.0
32    1.0
33    1.0
34    1.0
35    1.0
36    0.0
37    0.0
38    1.0
39    0.0
40    0.0
41    0.0
42    0.0
43    0.0
44    0.0
45    0.0
46    0.0
47    0.0
48    0.0
49    1.0
Name: LABEL_BaseExcess, dtype: float64

In [37]:
df = pd.DataFrame([[np.nan, 2, 1, 0],
                   [3, 4, np.nan, 1],
                   [1, np.nan, np.nan, np.nan],
                   [np.nan, 3, np.nan, 4]],
                  columns=list("ABCD"))
df.fillna(df.mean())

,A,B,C,D
0,2.0,2.0,1.0,0.000000
1,3.0,4.0,1.0,1.000000
2,1.0,3.0,1.0,1.666667
3,2.0,3.0,1.0,4.000000


5096


### Subtask1

In [46]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(criterion='gini', random_state=0)
X_train, X_test, y_train, y_test = train_test_split(train, labels1, test_size=0.2, random_state=42)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
y_pred_proba=clf.predict_proba(X_test)
display(y_pred, y_pred_proba, y_test)

array([[0., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

[array([[1., 0.],
        [0., 1.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]]),
 array([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]]),
 array([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [0., 1.],
        [1., 0.],
        [1., 0.]]),
 array([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [0., 1.],
        [1., 0.],
        [1., 0.]]),
 array([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [0., 1.],
        [1., 0.],
        [1., 0.]]),
 array([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]]),
 array([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]]),
 array([[1., 0.],
        [0., 1.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]]),
 array([[1., 0.]

,LABEL_BaseExcess,LABEL_Fibrinogen,LABEL_AST,LABEL_Alkalinephos,LABEL_Bilirubin_total,LABEL_Lactate,LABEL_TroponinI,LABEL_SaO2,LABEL_Bilirubin_direct,LABEL_EtCO2
2503,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
14148,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
2549,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0
42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5620,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
17131,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12919,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
7487,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16527,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [103]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
X_train, X_test, y_train, y_test = train_test_split(train, labels1, test_size=0.2, random_state=42)
#model = MultiOutputClassifier(RandomForestClassifier(n_estimators=1000))
clf=RandomForestClassifier(class_weight='balanced')
clf.fit(X_train, y_train)
y_pred_proba=clf.predict_proba(X_test)
#cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
#scores = cross_val_score(model, X_train, y_train, cv=cv, n_jobs=-1)
err2=roc_auc_score(y_test['LABEL_BaseExcess'], y_pred_proba[0][:,1])
# err=np.empty(10)
# TESTS = ['LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total',
#          'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2',
#          'LABEL_Bilirubin_direct', 'LABEL_EtCO2']
# for i,label in enumerate(TESTS):
#     err[i]=roc_auc_score(y_test[label], y_pred_proba[i][:,1])
# display(err)
display(err2)

0.5075343328319142

In [33]:
from sklearn.svm import SVC #run time 2.5 min. 
from sklearn.datasets import make_classification
from sklearn.metrics import roc_curve
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
X_train, X_test, y_train, y_test = train_test_split(train, labels1['LABEL_BaseExcess'], test_size=0.2, random_state=42)
#cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=1)
clf=SVC(kernel='rbf', class_weight='balanced', probability=True)
clf.fit(X_train, y_train)
#scores = cross_val_score(clf, X_test, y_test, scoring='roc_auc')
y_pred_proba=clf.predict_proba(X_train)
# fpr, tpr, thresholds = roc_curve(y_test, y_pred[])
# plt.plot(fpr, tpr)
#err=roc_auc_score(y_pred_proba[:,1], y_train)
# print(err)
display(y_pred_proba)

array([[0.71770794, 0.28229206],
       [0.74795707, 0.25204293],
       [0.74155712, 0.25844288],
       ...,
       [0.74186838, 0.25813162],
       [0.71825815, 0.28174185],
       [0.73373961, 0.26626039]])

In [19]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.datasets import make_classification
from sklearn.svm import SVC #run time 2.5 min. 
from sklearn.metrics import roc_curve
#X_train, X_test, y_train, y_test = train_test_split(train, labels1.iloc[:,0], test_size=0.2, random_state=42)
X_train, y_train = make_classification(n_samples=100, n_features=2, n_redundant=0, random_state=42)
base_clf=SVC(kernel='rbf', random_state=42)
calibrated_clf = CalibratedClassifierCV(base_estimator=base_clf, cv=3)
calibrated_clf.fit(X_train, y_train)
y_pred_proba=calibrated_clf.predict_proba(X_train)[:5, :]
display(y_pred_proba)
display(y_train)

array([[0.78631878, 0.21368122],
       [0.02671099, 0.97328901],
       [0.92511949, 0.07488051],
       [0.94312189, 0.05687811],
       [0.02339759, 0.97660241]])

array([0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0])

In [15]:
display(clf.predict(train))
display(labels1.iloc[:,0])

array([1., 0., 0., ..., 1., 1., 0.])

0        1.0
1        0.0
2        1.0
3        0.0
4        0.0
        ... 
18990    0.0
18991    0.0
18992    1.0
18993    0.0
18994    0.0
Name: LABEL_BaseExcess, Length: 18995, dtype: float64

In [42]:
from sklearn.svm import SVC 
from sklearn.metrics import roc_curve
sm = SMOTE(sampling_strategy=0.8,k_neighbors=5,random_state = 100)
X_train, X_test, y_train, y_test = train_test_split(train, labels1.iloc[:,2], test_size=0.2, random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)
clf=SVC(kernel='sigmoid', class_weight='balanced', probability=True)
clf.fit(X_train_res,y_train_res)
y_pred=clf.predict(X_test)
y_pred_proba =clf.predict_proba(X_test)
y_pred_proba=y_pred_proba[:,1]
display(y_pred_proba[0:30])

array([0.4463101 , 0.44589372, 0.44482212, 0.44543996, 0.44459013,
       0.44469117, 0.44612713, 0.44503015, 0.44508974, 0.44505348,
       0.48735781, 0.44538831, 0.44473464, 0.44678327, 0.44421893,
       0.44468597, 0.44541385, 0.44475473, 0.44805459, 0.44428501,
       0.44514118, 0.44459114, 0.44638775, 0.4447751 , 0.4451812 ,
       0.44502131, 0.44490617, 0.444677  , 0.44567095, 0.44463716])

In [43]:

# Oversample and plot imbalanced dataset with SMOTE
from collections import Counter
print(Counter(y_train))
print(Counter(y_train_res))
roc_auc_score(y_test, y_pred_proba, average=None)

Counter({0.0: 11559, 1.0: 3637})
Counter({0.0: 11559, 1.0: 9247})


0.4987384563458219

In [7]:
from sklearn.linear_model import RidgeClassifierCV
X_train, X_test, y_train, y_test = train_test_split(train, labels1.iloc[:,1], test_size=0.2, random_state=42)

clf = RidgeClassifierCV(alphas=(0.01,10,100), class_weight='balanced').fit(X_train, y_train)
roc_auc_score(y_test, clf.decision_function(X_test), average=None)

0.47242140693633017

In [8]:
#KNeighborsClassifier
#split into train and test set
X_train, X_test, y_train, y_test = train_test_split(train, labels1, test_size=0.2, random_state=42)
model = MultiOutputClassifier(KNeighborsClassifier())
model.fit(X_train, y_train)
y_pred_proba=model.predict_proba(X_test)
display(y_pred_proba)
y_pred_proba=y_pred_proba[:,1]
err=roc_auc_score(y_pred_proba, y_test)
print(err)
#y_pred.to_csv('submission.csv', index=False, header= None)

[array([[0.8, 0.2],
        [0.8, 0.2],
        [0.2, 0.8],
        ...,
        [0.6, 0.4],
        [0.6, 0.4],
        [0.8, 0.2]]),
 array([[0.6, 0.4],
        [0.8, 0.2],
        [0.6, 0.4],
        ...,
        [1. , 0. ],
        [1. , 0. ],
        [1. , 0. ]]),
 array([[0.8, 0.2],
        [1. , 0. ],
        [0.6, 0.4],
        ...,
        [0.8, 0.2],
        [0.6, 0.4],
        [0.6, 0.4]]),
 array([[0.8, 0.2],
        [1. , 0. ],
        [0.6, 0.4],
        ...,
        [0.8, 0.2],
        [0.6, 0.4],
        [0.6, 0.4]]),
 array([[0.8, 0.2],
        [1. , 0. ],
        [0.6, 0.4],
        ...,
        [0.8, 0.2],
        [0.6, 0.4],
        [0.6, 0.4]]),
 array([[1. , 0. ],
        [0.8, 0.2],
        [0.4, 0.6],
        ...,
        [1. , 0. ],
        [1. , 0. ],
        [0.8, 0.2]]),
 array([[0.8, 0.2],
        [1. , 0. ],
        [1. , 0. ],
        ...,
        [0.8, 0.2],
        [1. , 0. ],
        [1. , 0. ]]),
 array([[0.8, 0.2],
        [0.8, 0.2],
        [0.6, 0

TypeError: list indices must be integers or slices, not tuple

In [21]:
from sklearn.svm import SVC
# list_labels1=
# X_train, X_test, y_train, y_test = train_test_split(train, labels1, test_size=0.2, random_state=42)
model=SVC(probability=True)
list_probas=list()
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(train, labels1.iloc[:,i], test_size=0.2, random_state=42)
    model.fit(X_train, y_train)
    #y_pred = model.predict(X_test)
    #err=np.sqrt(mean_squared_error(y_pred,y_test))
    #print(model.predict_proba(X_test))
    list_probas.append(model.predict_proba(test))


In [56]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score

clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
list_probas=list()
y_pred=np.empty(10)
y_pred2=np.empty(10)
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(train, labels1.iloc[:,i], test_size=0.2, random_state=42)
    clf.fit(X_train, y_train)
    y_pred_ = clf.predict_proba(X_test)[:,1]
    err=roc_auc_score(y_pred_,y_test)
    y_pred[i]=err
    list_probas.append(clf.predict_proba(test))


ValueError: continuous format is not supported

In [10]:
print(y_pred)
print(y_pred2)

[0.52672709 0.28007445 0.4926413  0.48780873 0.49103572 0.45659163
 0.30868792 0.49210669 0.18780958 0.26311356]
[0.5027451  0.46162975 0.49039871 0.38182538 0.58831749 0.59116489
 0.55282024 0.40989828 0.4828722  0.46550816]


In [57]:
#GradientBoostingClassifier with SMOTE über alle Labels
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import GradientBoostingClassifier
sm = SMOTE(random_state=12)
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
list_probas=list()
y_pred=np.empty(10)
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(train, labels1.iloc[:,i], test_size=0.2, random_state=42)
    X_train_res, y_train_res = sm.fit_resample(X_train, y_train)
    clf.fit(X_train_res, y_train_res)
    y_pred_ = clf.predict(X_test)
    err=np.sqrt(mean_squared_error(y_pred_,y_test))
    err2=roc_auc_score(y_pred_,y_test)
    y_pred[i]=err
    y_pred2[i]=err2
    list_probas.append(clf.predict_proba(test))

In [58]:
display(y_pred2)

array([0.50126926, 0.5126165 , 0.51290521, 0.5138296 , 0.51653499,
       0.50940349, 0.50022901, 0.51934445, 0.49868486, 0.50393219])

In [9]:
#GradientBoostingClassifier with SMOTE über alle Labels
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import GradientBoostingClassifier
#sm = SMOTE(random_state=12)
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
list_probas=list()
y_pred=np.empty(10)
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(train, labels1.iloc[:,i], test_size=0.2, random_state=42)
    #X_train_res, y_train_res = sm.fit_resample(X_train, y_train)
    #clf.fit(X_train_res, y_train_res)
    clf.fit(X_train, y_train)
    y_pred_proba = clf.predict_proba(X_test)
    err=roc_auc_score(y_test,y_pred_proba[:,1])
    y_pred[i]=err
    #list_probas.append(clf.predict_proba(test))
display(y_pred)

array([0.50659956, 0.48425396, 0.49317427, 0.49325398, 0.50083773,
       0.49684827, 0.4867352 , 0.49814037, 0.44475019, 0.48489739])

In [65]:
#AdaBoostClassifier with SMOTE ein Label
from sklearn.ensemble import AdaBoostClassifier
from imblearn.over_sampling import SMOTE
import numpy as np
sm = SMOTE(random_state=12)

X_train, X_test, y_train, y_test = train_test_split(train, labels1.iloc[:,0], test_size=0.2, random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)
clf = AdaBoostClassifier(n_estimators=100, random_state=0)
clf.fit(X_train_res, y_train_res)
y_pred=clf.predict_proba(test)

display(y_pred)

array([[0.4969489 , 0.5030511 ],
       [0.49105561, 0.50894439],
       [0.49073774, 0.50926226],
       ...,
       [0.49863465, 0.50136535],
       [0.49694382, 0.50305618],
       [0.49511129, 0.50488871]])

In [3]:
from sklearn.kernel_ridge import KernelRidge
X_train, X_test, y_train, y_test = train_test_split(train, labels1.iloc[:,0], test_size=0.2, random_state=42)
kernel = KernelRidge(alpha=1.0)
kernel.fit(X_train, y_train)
y_pred=kernel.predict(X_test)
display(y_pred)



array([0.27250366, 0.26415627, 0.26743382, ..., 0.26791675, 0.2579573 ,
       0.25384091])

In [5]:
display(y_pred[0:100])

array([0.27250366, 0.26415627, 0.26743382, 0.29679835, 0.28718349,
       0.2695995 , 0.28129388, 0.26982355, 0.26295456, 0.26298779,
       0.28084609, 0.2552417 , 0.27333088, 0.2567275 , 0.27698311,
       0.25658484, 0.24511501, 0.26417031, 0.23048017, 0.25654252,
       0.28608089, 0.25176653, 0.26308138, 0.28504643, 0.25673986,
       0.30683147, 0.2634293 , 0.26688994, 0.25886158, 0.26967137,
       0.26538966, 0.28483164, 0.25932057, 0.26196864, 0.25969283,
       0.28435268, 0.32099809, 0.27625985, 0.27426806, 0.29058381,
       0.26134542, 0.25696133, 0.28624948, 0.26599748, 0.28566375,
       0.2450795 , 0.27073739, 0.25400278, 0.24731612, 0.2625042 ,
       0.24563452, 0.27613311, 0.27339991, 0.2553565 , 0.25675455,
       0.2616588 , 0.25289578, 0.26685504, 0.25992187, 0.26241302,
       0.27931201, 0.28870311, 0.24334527, 0.26979764, 0.27280393,
       0.26727779, 0.27409602, 0.31307404, 0.26170619, 0.27744151,
       0.26157485, 0.25634676, 0.27699079, 0.22711005, 0.31192

### Writing predicted values into submisson.csv

In [9]:
TESTS1 = ['LABEL_BaseExcess', 'LABEL_Fibrinogen']
TESTS = ['LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total',
         'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2',
         'LABEL_Bilirubin_direct', 'LABEL_EtCO2']
filename = 'sample.zip'
df_submission = pd.read_csv(filename)
for i,label in enumerate(TESTS):
    # round classification labels
    df_submission[label]=list_probas[i][:,0]
df_submission.to_csv('submission_smote.csv',index=False, header=False)

In [28]:
display(df_submission)

,pid,LABEL_BaseExcess,LABEL_Fibrinogen,LABEL_AST,LABEL_Alkalinephos,LABEL_Bilirubin_total,LABEL_Lactate,LABEL_TroponinI,LABEL_SaO2,LABEL_Bilirubin_direct,LABEL_EtCO2,LABEL_Sepsis,LABEL_RRate,LABEL_ABPm,LABEL_SpO2,LABEL_Heartrate
0,0,0.733201,0.926856,0.760559,0.763589,0.759297,0.801811,0.898982,0.767560,0.965157,0.934887,0.162,18.796,82.511,96.947,84.12
1,10001,0.733096,0.926974,0.760510,0.763614,0.759235,0.801788,0.899057,0.767406,0.965090,0.934566,0.486,18.796,82.511,96.947,84.12
2,10003,0.733020,0.926989,0.760479,0.763611,0.759176,0.801754,0.899065,0.767309,0.965086,0.934560,0.451,18.796,82.511,96.947,84.12
3,10004,0.719801,0.934951,0.757250,0.762114,0.752187,0.763668,0.882410,0.761005,0.971540,0.943170,0.785,18.796,82.511,96.947,84.12
4,10005,0.733050,0.926999,0.760519,0.763603,0.759223,0.801679,0.899053,0.767325,0.965076,0.934610,0.040,18.796,82.511,96.947,84.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12659,9989,0.733005,0.926999,0.760497,0.763601,0.759175,0.801694,0.899107,0.767252,0.965095,0.934523,0.647,18.796,82.511,96.947,84.12
12660,9991,0.732843,0.926986,0.760704,0.763681,0.759440,0.801470,0.898997,0.767729,0.965214,0.934999,0.916,18.796,82.511,96.947,84.12
12661,9992,0.733108,0.926977,0.760543,0.763643,0.759254,0.801752,0.898985,0.767344,0.965082,0.934577,0.093,18.796,82.511,96.947,84.12
12662,9994,0.732967,0.926994,0.760503,0.763593,0.759213,0.801470,0.899044,0.767225,0.965072,0.934559,0.936,18.796,82.511,96.947,84.12
